In [ ]:
# ============================================================
# Generate Full API Documentation (Markdown + PDF)
# Works with upgraded api_unified_data_full.json
# ============================================================

import json
from pathlib import Path
from xml.sax.saxutils import escape
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Preformatted, PageBreak
)
from reportlab.lib.units import inch
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
from reportlab.pdfbase import pdfmetrics

# -------------------
# Paths
# -------------------
INPUT_JSON = Path("api_unified_data_full.json")
PDF_PATH   = Path("API_Documentation.pdf")
MD_PATH    = Path("API_Documentation.md")

# -------------------
# Load Unified JSON
# -------------------
if not INPUT_JSON.exists():
    raise FileNotFoundError(f"Missing {INPUT_JSON}. Generate it first.")

with open(INPUT_JSON, "r", encoding="utf-8") as f:
    api_data = json.load(f)

# -------------------
# Register Unicode font (for multi-language text)
# -------------------
# Note: HeiseiMin-W3 comes from ReportLab Asian font pack (CID fonts).
# If not installed in your environment, replace with "Helvetica".
try:
    pdfmetrics.registerFont(UnicodeCIDFont("HeiseiMin-W3"))
    BASE_FONT = "HeiseiMin-W3"
except Exception:
    BASE_FONT = "Helvetica"

# -------------------
# PDF Styles
# -------------------
styles = getSampleStyleSheet()

# Avoid overriding built-in style names; create new ones.
styles.add(ParagraphStyle(
    name="H1Style", fontSize=16, leading=20, spaceAfter=10,
    fontName=BASE_FONT, textColor=colors.darkblue
))
styles.add(ParagraphStyle(
    name="H2Style", fontSize=13, leading=18, spaceAfter=8,
    fontName=BASE_FONT, textColor=colors.darkgreen
))
styles.add(ParagraphStyle(
    name="BodyStyle", fontSize=10, leading=14, fontName=BASE_FONT
))
styles.add(ParagraphStyle(
    name="CodeStyle", fontSize=8, fontName="Courier", leading=11,
    backColor=colors.whitesmoke, spaceBefore=4, spaceAfter=4
))

# -------------------
# Helpers
# -------------------
def para(text: str, style_name="BodyStyle") -> Paragraph:
    """
    Safe Paragraph creation with XML escaping.
    Converts newlines to <br/> for nicer paragraphs.
    """
    if text is None:
        text = ""
    text = str(text)
    # Escape &, <, > and keep basic reportlab inline tags if you add them later.
    text = escape(text).replace("\n", "<br/>")
    return Paragraph(text, styles[style_name])

def code_block(text: str) -> Preformatted:
    """Monospaced preformatted block (preserves whitespace)."""
    text = "" if text is None else str(text)
    return Preformatted(text, styles["CodeStyle"])

def schema_block(title: str, text: str):
    """
    A labeled schema section.
    Expects 'text' to already be a printable string (e.g., schema_text).
    """
    chunks = []
    chunks.append(para(f"{title}:", "H2Style"))
    if text and str(text).strip():
        chunks.append(code_block(text))
    else:
        chunks.append(para("_None_", "BodyStyle"))
    return chunks

def parameter_table(params):
    """Convert parameters to a table using schema_flat/type when available."""
    if not params:
        return [para("_No parameters defined._")]

    data = [["Name", "In", "Type", "Required", "Description"]]
    for p in params:
        p_name = p.get("name", "")
        p_in = p.get("in", "")
        p_required = str(p.get("required", False))
        p_desc = p.get("description", "")

        # Prefer 'schema_flat.type'; fallback to parse first line of schema_text.
        schema_flat = p.get("schema_flat") or {}
        p_type = schema_flat.get("type")
        if not p_type:
            schema_text = p.get("schema_text") or ""
            # Attempt to extract from first "- Type: XXX" line.
            for line in str(schema_text).splitlines():
                s = line.strip()
                if s.startswith("- Type:"):
                    p_type = s.replace("- Type:", "").strip()
                    break
        if not p_type:
            p_type = "—"

        data.append([p_name, p_in, p_type, p_required, p_desc])

    # Reasonable widths for A4 with ~36pt margins each side (usable width ~523pt)
    table = Table(data, repeatRows=1, colWidths=[100, 60, 90, 60, 200])
    table.setStyle(TableStyle([
        ("BACKGROUND", (0, 0), (-1, 0), colors.lightgrey),
        ("TEXTCOLOR", (0, 0), (-1, 0), colors.black),
        ("ALIGN", (0, 0), (-1, -1), "LEFT"),
        ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
        ("FONTSIZE", (0, 0), (-1, -1), 8),
        ("INNERGRID", (0, 0), (-1, -1), 0.25, colors.grey),
        ("BOX", (0, 0), (-1, -1), 0.25, colors.grey),
        ("VALIGN", (0, 0), (-1, -1), "TOP"),
    ]))
    return [table]

def workflow_section(api_entry):
    """Show workflow / next calls based on 'calls_next' list."""
    calls = api_entry.get("calls_next", []) or []
    if not calls:
        return [para("_No dependent API calls._")]
    bullets = "\n".join([f"• {c}" for c in calls])
    return [para(bullets)]

# -------------------
# Build Markdown
# -------------------
md_lines = []
md_lines.append("# 🧾 Agentic API Workflow Assistant\n")
md_lines.append("**Comprehensive API Reference Document**\n")
md_lines.append("This document provides API details including parameters, request/response schemas, and workflow relationships. Generated automatically from Swagger/OpenAPI JSON.\n")

for i, api in enumerate(api_data, 1):
    title = api.get("summary") or api.get("path") or f"Endpoint {i}"
    path = api.get("path", "")
    method = api.get("method", "")
    op_id = api.get("operation_id", "-")
    desc = api.get("description", "")

    md_lines.append(f"\n---\n\n## {i}. {title}\n")
    md_lines.append(f"**Path:** `{path}`  \n**Method:** `{method}`  \n**Operation ID:** `{op_id}`\n")
    if desc:
        md_lines.append(f"\n{desc}\n")

    # Parameters
    md_lines.append("\n**Parameters**\n")
    params = api.get("parameters", []) or []
    if params:
        md_lines.append("| Name | In | Type | Required | Description |")
        md_lines.append("|---|---|---|---|---|")
        for p in params:
            nm = p.get("name","")
            pin = p.get("in","")
            req = str(p.get("required", False))
            pdesc = (p.get("description","") or "").replace("\n"," ")
            ptype = p.get("schema_flat", {}).get("type") or ""
            if not ptype:
                st = p.get("schema_text","") or ""
                for line in st.splitlines():
                    s = line.strip()
                    if s.startswith("- Type:"):
                        ptype = s.replace("- Type:","").strip()
                        break
            md_lines.append(f"| {nm} | {pin} | {ptype or '—'} | {req} | {pdesc} |")
    else:
        md_lines.append("_No parameters defined._")

    # Request schema
    req_text = ""
    if "request_body" in api:
        req_text = api["request_body"].get("schema_text","") or ""
    elif "request_schema" in api:
        # Backward compatibility with the older extractor
        req_text = api.get("request_schema","") or ""

    if req_text.strip():
        md_lines.append("\n**Request Schema**\n")
        md_lines.append("```text")
        md_lines.append(req_text)
        md_lines.append("```")
    else:
        md_lines.append("\n**Request Schema**: _None_")

    # Response schema
    resp_text = api.get("responses_schema_text","") or api.get("response_schema","") or ""
    if resp_text.strip():
        md_lines.append("\n**Response Schema**\n")
        md_lines.append("```text")
        md_lines.append(resp_text)
        md_lines.append("```")
    else:
        md_lines.append("\n**Response Schema**: _None_")

    # Workflow
    calls = api.get("calls_next",[]) or []
    md_lines.append("\n**Workflow / Next API Calls**\n")
    if calls:
        for c in calls:
            md_lines.append(f"- {c}")
    else:
        md_lines.append("_No dependent API calls._")

# Write Markdown
MD_PATH.write_text("\n".join(md_lines), encoding="utf-8")

# -------------------
# Build PDF Document
# -------------------
doc = SimpleDocTemplate(
    str(PDF_PATH),
    pagesize=A4,
    rightMargin=36, leftMargin=36, topMargin=36, bottomMargin=36
)

content = []

# Title page
content.append(para("🧾 Agentic API Workflow Assistant", "H1Style"))
content.append(para("Comprehensive API Reference Document", "BodyStyle"))
content.append(Spacer(1, 10))
content.append(para("This document provides API details including parameters, request/response schemas, and workflow relationships. Generated automatically from Swagger/OpenAPI JSON.", "BodyStyle"))
content.append(Spacer(1, 20))

for i, api in enumerate(api_data, 1):
    title = api.get("summary") or api.get("path") or f"Endpoint {i}"
    path = api.get("path", "")
    method = api.get("method", "")
    op_id = api.get("operation_id", "-")
    desc = api.get("description", "")

    # Section header
    content.append(para(f"{i}. {title}", "H1Style"))
    content.append(para(f"Path: {path}\nMethod: {method}\nOperation ID: {op_id}", "BodyStyle"))
    content.append(Spacer(1, 6))
    if desc:
        content.append(para(desc, "BodyStyle"))
        content.append(Spacer(1, 6))

    # Parameters
    content.append(para("Parameters", "H2Style"))
    content.extend(parameter_table(api.get("parameters", [])))
    content.append(Spacer(1, 8))

    # Request Schema
    if "request_body" in api:
        req_text = api["request_body"].get("schema_text","") or ""
    else:
        # Backward compatibility with earlier extractor field name
        req_text = api.get("request_schema","") or ""
    content.extend(schema_block("Request Schema", req_text))
    content.append(Spacer(1, 8))

    # Response Schema (consolidated)
    resp_text = api.get("responses_schema_text","") or api.get("response_schema","") or ""
    content.extend(schema_block("Response Schema", resp_text))
    content.append(Spacer(1, 8))

    # Workflow / Next calls
    content.append(para("Workflow / Next API Calls", "H2Style"))
    content.extend(workflow_section(api))
    content.append(Spacer(1, 14))

    # Optional page break every ~10 endpoints to keep file readable
    if i % 10 == 0:
        content.append(PageBreak())
print(content)
# Build the PDF
doc.build(content)

print(f"✅ PDF documentation generated successfully → {PDF_PATH.resolve()}")
print(f"✅ Markdown documentation generated successfully → {MD_PATH.resolve()}")
print(content)



In [12]:
with open("documentation.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(content))